# Ubicación de oficina para startup

### Importación de librerías que pudiera precisar...

In [1]:
import pandas as pd
import numpy as np
import folium
from folium import plugins
import pymongo
from pandas.io.json import json_normalize

#### 1.- Descripción cliente en general...
1.- Mi cliente busca atender a distintas empresas de manufactura desarrollando software de tipo empresarial para sensores y
mediciones.
2.- Quiere basarse en la concentración de empresas asentadas cerca de las zonas urbanas que pudieran interesarle.
3.- Le gustaría intentar asentarse en el medio Este o en el centro-sur de los Estados Unidos.

In [2]:
#Conexión a la base de datos en DBMongo...
from pymongo import MongoClient
client=MongoClient('mongodb://localhost:27017')

In [3]:
#Lectura de la base de datos...
db=client.companies

In [4]:
data=db.data

#### 2.- Filtrar la base de datos...

In [5]:
#Hay que deshacernos de las empresas que ya se hayan depreciado...
activas=db.companies.find({'deadpooled_year':{'$eq':None}},
                                 {'name':1, '_id':0,'category_code':1,'offices':1,
                                  'ipo.valuation_amount':1,'offices[0].state_code':1})

In [6]:
df=pd.DataFrame(activas)
df.head()

,name,category_code,offices,ipo
0,Digg,news,"[{'description': None, 'address1': '135 Missis...",NaN
1,Facebook,social,"[{'description': 'Headquarters', 'address1': '...",{'valuation_amount': 104000000000}
2,Postini,web,"[{'description': None, 'address1': '959 Skyway...",NaN
3,Geni,web,"[{'description': 'Headquarters', 'address1': '...",NaN
4,Flektor,games_video,"[{'description': None, 'address1': '8536 Natio...",NaN


In [7]:
#Vemos la cantidad de compañías que aún trabajan...
df.shape
#Son un total de 17872 empresas, entre pequeñas, medianas y grandes.

(17872, 4)

In [8]:
#Ahora es preciso revisar a qué se dedican
activities=set(df.category_code.unique())
print(activities)
print(len(activities))
#Son 42 campos de actividad de empresas activas entre chicas, medianas y grandes..

{'social', 'messaging', 'transportation', 'nanotech', 'real_estate', 'hospitality', 'public_relations', 'finance', 'government', 'analytics', 'enterprise', 'semiconductor', 'sports', 'health', 'biotech', 'medical', 'cleantech', 'advertising', 'legal', 'education', 'ecommerce', 'network_hosting', 'local', None, 'hardware', 'photo_video', 'fashion', 'automotive', 'music', 'manufacturing', 'security', 'mobile', 'design', 'search', 'news', 'nonprofit', 'software', 'games_video', 'other', 'travel', 'consulting', 'web'}
42


In [9]:
#Hay que conseguir las ubicaciones de éstas empresas, por lo menos de la oficina 0... Como nos explicó Yonatan
oficinas=db.companies.find({'$and': [{'deadpooled_year':{'$eq':None}},\
                                     {'offices':{'$not':{'$size':0}}},{'category_code':{'$eq':'manufacturing'}}]},\
                                     {'name':1, '_id':0,'category_code':1,'offices':1,\
                                      'ipo.valuation_amount':1})

oficinas_df=pd.DataFrame(oficinas)
oficinas_df.head()

,name,category_code,offices,ipo
0,Raydiance,manufacturing,"[{'description': '', 'address1': 'Chevy Chase ...",NaN
1,SmartEquip,manufacturing,"[{'description': 'Headquarters', 'address1': '...",NaN
2,Honeywell,manufacturing,"[{'description': 'World Headquarters', 'addres...",{'valuation_amount': None}
3,Skycross,manufacturing,"[{'description': '', 'address1': '7341 Office ...",NaN
4,Solyndra,manufacturing,"[{'description': '', 'address1': '47488 Kato R...",NaN


In [10]:
oficinas_df.shape

(19, 4)

In [11]:
#Uso la rutina de Yonatan para encontrar las oficinas "centrales" de las empresas de manufactura
#que me quedan de la base de datos.
def get_first(data):
    data=data['offices']
    
    principal=None
    
    if data[0]['latitude'] and data[0]['longitude']:
        principal={'type':'Point',
                  'coordinates':[data[0]['longitude'],
                                 data[0]['latitude']]}
        
    return {'totalOffices':len(data),
           'lat':data[0]['latitude'],
           'lng':data[0]['longitude'],
            'oficina_principal':principal}

In [12]:
first_office=oficinas_df[['offices']].apply(get_first, result_type='expand', axis=1)

In [13]:
first_office

,totalOffices,lat,lng,oficina_principal
0,1.0,38.965240,-77.083389,"{'type': 'Point', 'coordinates': [-77.083389, ..."
1,2.0,41.143608,-73.427439,"{'type': 'Point', 'coordinates': [-73.427439, ..."
2,1.0,40.795862,-74.455515,"{'type': 'Point', 'coordinates': [-74.455515, ..."
3,1.0,NaN,NaN,NaN
4,1.0,37.475721,-121.931712,"{'type': 'Point', 'coordinates': [-121.9317123..."
5,1.0,37.371138,-121.998365,"{'type': 'Point', 'coordinates': [-121.998365,..."
6,1.0,33.325335,-111.859162,"{'type': 'Point', 'coordinates': [-111.8591619..."
7,1.0,29.539901,-95.064003,"{'type': 'Point', 'coordinates': [-95.064003, ..."
8,1.0,NaN,NaN,NaN
9,1.0,37.383175,-121.915418,"{'type': 'Point', 'coordinates': [-121.915418,..."


In [14]:
#Tiramos las que tienen Nan para ver de forma efectiva cuántas quedan...
first_office=first_office.dropna()

In [15]:
first_office.head()

,totalOffices,lat,lng,oficina_principal
0,1.0,38.965240,-77.083389,"{'type': 'Point', 'coordinates': [-77.083389, ..."
1,2.0,41.143608,-73.427439,"{'type': 'Point', 'coordinates': [-73.427439, ..."
2,1.0,40.795862,-74.455515,"{'type': 'Point', 'coordinates': [-74.455515, ..."
4,1.0,37.475721,-121.931712,"{'type': 'Point', 'coordinates': [-121.9317123..."
5,1.0,37.371138,-121.998365,"{'type': 'Point', 'coordinates': [-121.998365,..."


In [16]:
first_office.shape
#Son pocas ubicaciones, se generará mapa para verificar la ubicación de forma inicial...

(13, 4)

In [17]:
df3=first_office[['lat','lng']].values
map_us=folium.Map(location=[39.359337,-99.415253])
map_us.add_child(plugins.HeatMap(df3))


#### De acuerdo con el mapa, la mancha de calor inicialmente indicaría que la concentración más grande de empresas de manufactura que podría interesar a mio cliente es en la costa Este de los Estados Unidos. Sin embargo en un caso realista, sabemos que en los Estados Unidos hay miles de empresas manufactureras de diversa índole.

#### Para hacer el ejemplo un poco más realista, voy a intentar extraer datos en un archivo kml de Googlemaps para aumentar la cantidad de puntos de empresas manufactureras, aunque sea como unos pocos ejemplos....

### De Googlemaps obtuve una serie de ubicaciones aproximadas de diversas empresas manufactureras de diversa índole, y exporté un archivo kmz. De lo que he revisado, un archivo kmz es un *.zip que tiene un doc.kml, así como un folder con las imágenes e íconos que hacen referencia a éste, tomaré doc.kml, que es el archivo dentro del kmz para extraer solo las ubicaciones e insertarlas en el dataframe que ya tenemos para ver cómo cambia el heatmap de potenciales ciudades para la ubicación de su startup...

In [22]:
#Obtengo solo las ubicaciones para no tener problemas en juntar los nuevos datos del archiv kml que obtuve de googlemaps...
df3 = pd.DataFrame(zip(first_office.lat, first_office.lng))
df3.rename(columns={0: "lat", 1: "lng"}, inplace=True)
df3.head()

,lat,lng
0,38.965240,-77.083389
1,41.143608,-73.427439
2,40.795862,-74.455515
3,37.475721,-121.931712
4,37.371138,-121.998365


In [ ]:
# Traté de usar esta librería de opengis, pero no me ha permitido instalarla en jupyter de forma correcta, por lo que lo haré
#de manera directa para leer un xml...
#pip install gdal .... Este no está descargando correctamente

In [18]:
#from osgeo import ogr

#ds = ogr.Open('doc.kml')

#for lyr in ds:
#    for feat in lyr:
#        geom = feat.GetGeometryRef()
#        if geom != None:
#            for i in range(0, geom.GetPointCount()):
#                print (geom.GetPoint(i))

ModuleNotFoundError: No module named 'osgeo'

In [84]:
#pip install simplekml

  Created wheel for simplekml: filename=simplekml-1.3.3-cp37-none-any.whl size=51388 sha256=ebc7c16f4558758bf9abee7ea8d80b1e9749748106e537cc4ea6ccac215adbbd
  Stored in directory: C:\Users\Arturo\AppData\Local\pip\Cache\wheels\16\e6\f2\4f58e4252de73f6931fc64a013f290c9a9c285631ba8f49ac3
Successfully built simplekml
Note: you may need to restart the kernel to use updated packages.


In [23]:
#Importamos BS4 y csv
from bs4 import BeautifulSoup
import csv

def process_coordinate_string(str):

#Se efectúa la lectura de archivo de los datos de localización y se hace el split de ambas coordenadas geográficas..

    ret = []
    comma_split = str.split(',')
    return [comma_split[1], comma_split[0]]

def main():

#Abrimos el kml y vamos leyendo renglón por renglón...

    with open('doc.kml', 'r') as f:
        s = BeautifulSoup(f, 'xml')
        with open('out.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            for coords in s.find_all('coordinates'):
                writer.writerow(process_coordinate_string(coords.string))

main()

In [24]:
df4 = pd.read_csv("out.csv")
# La verdad no recordé incluir los encabezados en el csv y ya mejor para terminar pronto el ejemplo, cambié solo el
# nombre de columnas... Sorry :S
df4.rename(columns={'37.2321108': "lat", '\n            -121.7804504': "lng"}, inplace=True)
df4.head()

,lat,lng
0,39.884640,-86.249519
1,25.887089,-80.209735
2,37.067828,-84.988804
3,31.497672,-97.198261
4,36.041366,-79.958070


In [25]:
df5=df3.append(df4[df4.isin(df3) == False])
df5.shape
#Quedan un total de 50 ubicaciones que pudeiran serivr como referencia general.

(52, 2)

In [26]:
#Generamos el nuevo mapa de calor con las ubicaciones de las empresas que ya tenemos en total...
map_us2=folium.Map(location=[39.359337,-99.415253])
map_us2.add_child(plugins.HeatMap(df5))

### Ya se oberva una mejora en el mapa de calor 🔥  y podemos seleccionar tres opciones

In [28]:
map_us3=folium.Map(location=[39.359337,-99.415253])
folium.CircleMarker([41.484743, -81.708414], radius=20, on=folium.Icon(), line_color='#92000a').add_to(map_us3)
folium.CircleMarker([37.428093, -121.903618], radius=20, on=folium.Icon(), line_color='#92000a').add_to(map_us3)
folium.CircleMarker([30.242966, -97.748338], radius=20, on=folium.Icon(), line_color='#92000a').add_to(map_us3)
map_us3.add_child(plugins.HeatMap(df5))

### Las ciudades que quedarían como opciones viables son: San josé, California; Austin, Texas; y Cleveland, Ohio, sin embargo, aún falta hacer el análisis detallado de las distancias ya que sea elegida una ciudad de las tres mecionadas.

In [ ]:
#Aqui faltaría el análisis de distancias ya selecta una ciudad....
#Preciso rehacer la lectura del csv para incluir nombres de las empresas que incluí en el dataframe como adicionales....
#

### Generación de kml de prueba...

In [29]:
#Genero un kml de prueba para ver si funciona con puntos varios, 
import simplekml

kml = simplekml.Kml()
latitudes=df5['lat'].tolist()
longitudes=df5['lng'].tolist()

for i in range(len(latitudes)):
    pnt=kml.newpoint(name="Manuf", coords=[(longitudes[i],latitudes[i])])  # lon, lat, optional height

kml.save("Manufacturing_US_F.kml")
#Funciona bien :)#